# Import Hook

In [1]:
%load_ext literary.module

In [2]:
import sys
from pathlib import Path

from ..notebook.importer import NotebookImporter
from ..notebook.patch import patch

Here we implement a `ModuleImporter` class. 

In [3]:
class ModuleImporter(NotebookImporter):
    pass

We need to be able to determine the "package name" from a notebook path.

In [4]:
@patch(ModuleImporter)
def determine_package_name(self, path: Path) -> str:
    """Determine the corresponding importable name for a package directory given by
    a particular file path. Return `None` if path is not contained within `sys.path`.

    :param path: path to package
    :return:
    """
    for p in sys.path:
        # If the cwd is only on `sys.path` exactly, then it can't
        # be a submodule. However, let's first look for parent paths
        if str(path) == p:
            continue

        # Resolve path relative to `sys.path`
        try:
            relative_path = path.relative_to(p)
        except ValueError:
            continue

        return ".".join(relative_path.parts)

    # Nothing was found on `sys.path`, so we abort
    return None

Before installing the hook, we should remove the CWD from the `PATH`.

In [5]:
@patch(ModuleImporter)
def install_hook(self):
    """Install notebook import hook

    Don't allow the user to specify a custom search path, because we also need this to
    interoperate with the default Python module importers which use sys.path

    :return:
    """
    cwd = Path.cwd()
    sys.path = [p for p in sys.path if Path(p).resolve() != cwd]
    
    super().install_hook()

When updating the user namespace, we define the current `__package__`

In [8]:
@patch(ModuleImporter)
def update_namespace(self, namespace):
    cwd = Path.cwd()

    # Update user namespace
    namespace.update(
        {
            "__package__": self.determine_package_name(cwd),
            "patch": patch,
        }
    )